# Data Cleaning, Python 3

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
# import data
tap = pd.read_csv("TAP_data.csv")
print(tap.shape)
nrows = tap.shape[0]
ncols = tap.shape[1]

(190964, 17)


From the above output, our dataset has 190,964 examples across 17 features.

In [36]:
tap.head()

,Academic Year,Level,TAP Level of Study,Sector Type,TAP Sector Group,Recipient Age Group,TAP Financial Status,TAP Award Schedule,TAP Degree or NonDegree,TAP Schedule Letter,"Income by $1,000 Range","Income by $5,000 Range","Income by $10,000 Range",TAP Recipient Headcount,TAP Recipient FTEs,TAP Recipient Dollars,Award per FTE
0,2015,U,4 yr Undergrad,PRIVATE,6-BUS. DEGREE,age 22 - 25,Financial_Dependent,Dependent_Schedule,Degree,E,"$ 1,001 to $ 2,000","$ 1 to $ 5,000","$ 0 to $10,000",23,18.92,"$96,170.20","$5,082.99"
1,2015,U,2 yr Undergrad,PRIVATE,5-INDEPENDENT,age 26 - 35,Financial_Independent,Independent_Schedule,Degree,M,"$ 6,001 to $ 7,000","$ 5,001 to $10,000","$ 0 to $10,000",10,6.20,"$11,802.47","$1,903.62"
2,2015,U,4 yr Undergrad,PRIVATE,6-BUS. DEGREE,age 26 - 35,Financial_Dependent,Dependent_Schedule,Degree,E,"$ 2,001 to $ 3,000","$ 1 to $ 5,000","$ 0 to $10,000",1,0.17,$847.96,"$4,988.00"
3,2015,U,2 yr Undergrad,PRIVATE,5-INDEPENDENT,under age 22,Financial_Dependent,Dependent_Schedule,Degree,E,"$12,001 to $13,000","$10,001 to $15,000","$10,001 to $20,000",12,10.30,"$48,704.45","$4,728.59"
4,2015,U,2 yr Undergrad,PRIVATE,5-INDEPENDENT,age 26 - 35,Financial_Independent,Dependent_Schedule,Degree,E,"$25,001 to $26,000","$25,001 to $30,000","$20,001 to $30,000",2,1.51,"$4,951.94","$3,279.43"


We rename the columns for easier manipulation:

In [37]:
features = list(tap.columns.values)
print(features)

['Academic Year', 'Level', 'TAP Level of Study', 'Sector Type', 'TAP Sector Group', 'Recipient Age Group', 'TAP Financial Status', 'TAP Award Schedule', 'TAP Degree or NonDegree', 'TAP Schedule Letter', 'Income by $1,000 Range', 'Income by $5,000 Range', 'Income by $10,000 Range', 'TAP Recipient  Headcount', 'TAP Recipient FTEs', 'TAP Recipient Dollars', 'Award per FTE']


In [38]:
tap_renamed = tap.rename(columns={'Academic Year': 'year', 'Level': 'undergrad', 'TAP Level of Study' : 'level', 'Sector Type' : 'type', 'TAP Sector Group' : 'group', 'Recipient Age Group' : 'age' , 'TAP Financial Status' : 'dependent', 'TAP Award Schedule' : 'schedule', 'TAP Degree or NonDegree' : 'degree', 'TAP Schedule Letter' : 'schedule letter' , 'Income by $1,000 Range': 'income_1', 'Income by $5,000 Range' : 'income_5', 'Income by $10,000 Range' : 'income_10', 'TAP Recipient  Headcount' : 'headcount', 'TAP Recipient FTEs' : 'fte', 'TAP Recipient Dollars' : 'dollars', 'Award per FTE': 'award_per_fte' })

In [39]:
renamed_features = list(tap_renamed.columns.values)
print(renamed_features)

['year', 'undergrad', 'level', 'type', 'group', 'age', 'dependent', 'schedule', 'degree', 'schedule letter', 'income_1', 'income_5', 'income_10', 'headcount', 'fte', 'dollars', 'award_per_fte']


Look at data types:

In [40]:
tap.dtypes

Academic Year                 int64
Level                        object
TAP Level of Study           object
Sector Type                  object
TAP Sector Group             object
Recipient Age Group          object
TAP Financial Status         object
TAP Award Schedule           object
TAP Degree or NonDegree      object
TAP Schedule Letter          object
Income by $1,000 Range       object
Income by $5,000 Range       object
Income by $10,000 Range      object
TAP Recipient  Headcount      int64
TAP Recipient FTEs          float64
TAP Recipient Dollars        object
Award per FTE                object
dtype: object

Observe unique values in each feature:

In [41]:
for j in list(tap.columns.values):
    if tap[j].dtypes == "object" and j != "TAP Recipient Dollars" and j != "Award per FTE" and j != "Income by $1,000 Range" and j != "Income by $5,000 Range" and j != "Income by $10,000 Range":
        print(j)
        print(tap[j].unique())

Level
['U' 'G']
TAP Level of Study
['4 yr Undergrad' '2 yr Undergrad' '5 yr Undergrad' 'STAP' 'Grad']
Sector Type
['PRIVATE' 'PUBLIC']
TAP Sector Group
['6-BUS. DEGREE' '5-INDEPENDENT' '3-SUNY SO' '8-OTHER' '7-BUS. NON-DEG'
 '4-SUNY CC' '1-CUNY SR' '2-CUNY CC' '9-CHAPTER XXII']
Recipient Age Group
['age 22 - 25' 'age 26 - 35' 'under age 22' 'age 36 - 50' 'over age 50']
TAP Financial Status
['Financial_Dependent' 'Financial_Independent']
TAP Award Schedule
['Dependent_Schedule' 'Independent_Schedule' 'Married_No_Dependents']
TAP Degree or NonDegree
['Degree' 'Non_Degree']
TAP Schedule Letter
['E' 'M' 'C' 'A' 'D' 'K' 'L' 'H' 'P' 'G' 'F' 'V' 'U']


## Transforming data

In [42]:
tap_clean = tap_renamed

### Binary feature encoding

We first want to modify the Level feature, which indicates whether the application was from an undergraduate ('U') or graduate ('T'). We set the value to 1 if undergraduate, 0 otherwise.

We do the same for the other binary features:
* sector type: 1 for Private and 0 for Public
* dependent: 1 for Dependent, 0 for Independent
* degree: 1 for Degree, 0 for NonDegree

In [43]:
tap_clean['undergrad'] = tap_clean['undergrad'].str.strip()
tap_clean['undergrad'] = tap_clean['undergrad'].apply(lambda x: 1 if x == 'U' else 0)

tap_clean['type'] = tap_clean['type'].str.strip()
tap_clean['type'] = tap_clean['type'].apply(lambda x: 1 if x == 'PRIVATE' else 0)

tap_clean['dependent'] = tap_clean['dependent'].str.strip()
tap_clean['dependent'] = tap_clean['dependent'].apply(lambda x: 1 if x == 'Financial_Dependent' else 0)

tap_clean['degree'] = tap_clean['degree'].str.strip()
tap_clean['degree'] = tap_clean['degree'].apply(lambda x: 1 if x == 'Degree' else 0)

In [44]:
tap_clean.head()

,year,undergrad,level,type,group,age,dependent,schedule,degree,schedule letter,income_1,income_5,income_10,headcount,fte,dollars,award_per_fte
0,2015,1,4 yr Undergrad,1,6-BUS. DEGREE,age 22 - 25,1,Dependent_Schedule,1,E,"$ 1,001 to $ 2,000","$ 1 to $ 5,000","$ 0 to $10,000",23,18.92,"$96,170.20","$5,082.99"
1,2015,1,2 yr Undergrad,1,5-INDEPENDENT,age 26 - 35,0,Independent_Schedule,1,M,"$ 6,001 to $ 7,000","$ 5,001 to $10,000","$ 0 to $10,000",10,6.20,"$11,802.47","$1,903.62"
2,2015,1,4 yr Undergrad,1,6-BUS. DEGREE,age 26 - 35,1,Dependent_Schedule,1,E,"$ 2,001 to $ 3,000","$ 1 to $ 5,000","$ 0 to $10,000",1,0.17,$847.96,"$4,988.00"
3,2015,1,2 yr Undergrad,1,5-INDEPENDENT,under age 22,1,Dependent_Schedule,1,E,"$12,001 to $13,000","$10,001 to $15,000","$10,001 to $20,000",12,10.30,"$48,704.45","$4,728.59"
4,2015,1,2 yr Undergrad,1,5-INDEPENDENT,age 26 - 35,0,Dependent_Schedule,1,E,"$25,001 to $26,000","$25,001 to $30,000","$20,001 to $30,000",2,1.51,"$4,951.94","$3,279.43"


### Categorial feature transformation via one-hot encoding

**TAP Level of Study**

The TAP Level of Study feature indicates the applicant's current level of academic studies: 2 year undergrad, 4 year undergrad, 5 year undergrad, graduate school, or STAP (Supplemental Tuition Assistance Program), which is a form of authorized aid for remedial courses.

We clean the data via one-hot encoding below.

In [45]:
# clean the level column and transform with one-hot encoding
print(tap_clean.groupby('level').size())
print(np.sum(tap_clean.groupby('level').size()))

level
2 yr Undergrad    93450
4 yr Undergrad    80888
5 yr Undergrad    10733
Grad               5690
STAP                203
dtype: int64
190964


In [46]:
# one-hot encode for TAP level of study
tap_clean['level'] = tap_clean['level'].str.strip()

tap_clean['level_2'] = tap_clean['level'].apply(lambda x: 1 if x == '2 yr Undergrad' else 0) # for 2 year undergrad
tap_clean['level_4'] = tap_clean['level'].apply(lambda x: 1 if x == '4 yr Undergrad' else 0) # for 4 year undergrad
tap_clean['level_5'] = tap_clean['level'].apply(lambda x: 1 if x == '5 yr Undergrad' else 0) # for 5 year undergrad
tap_clean['level_g'] = tap_clean['level'].apply(lambda x: 1 if x == 'Grad' else 0) # for graduate
tap_clean['level_s'] = tap_clean['level'].apply(lambda x: 1 if x == 'STAP' else 0) # for STAP

np.sum(tap_clean['level_2']==1) + np.sum(tap_clean['level_4']==1) + np.sum(tap_clean['level_5']==1) + np.sum(tap_clean['level_g']==1) + np.sum(tap_clean['level_s']==1)

190964

**TAP Sector Group**

The TAP Sector Group feature encodes for the type of institution within New York state that the application is for: 
* 1-CUNY SR = CUNY Senior Colleges 
* 2-CUNY CC = CUNY Community Colleges 
* 3-SUNY SO = SUNY State Operated 
* 4-SUNY CC = SUNY Community Colleges 
* 5-INDEPENDENT = Independent Colleges 
* 6-BUS. DEGREE = Business Degree Granting Institutions 
* 7-BUS. NON-DEG = Non-Degree Business Schools 
* 8-OTHER = All Other Institutions 
* 9-CHAPTER XXII = Chapter XXII TAP Schools

Again, like above, we one-hot encode for the sector group.

In [47]:
# clean the level column and transform with one-hot encoding
print(tap_clean.groupby('group').size())
print(np.sum(tap_clean.groupby('group').size()))

group
1-CUNY SR         28477
2-CUNY CC         13686
3-SUNY SO         40899
4-SUNY CC         22744
5-INDEPENDENT     42618
6-BUS. DEGREE     30544
7-BUS. NON-DEG     5353
8-OTHER            5414
9-CHAPTER XXII     1229
dtype: int64
190964


In [48]:
# one-hot encode for TAP sector group
tap_clean['group'] = tap_clean['group'].str.strip()

tap_clean['group_1'] = tap_clean['group'].apply(lambda x: 1 if x == '1-CUNY SR' else 0)
tap_clean['group_2'] = tap_clean['group'].apply(lambda x: 1 if x == '2-CUNY CC' else 0)
tap_clean['group_3'] = tap_clean['group'].apply(lambda x: 1 if x == '3-SUNY SO' else 0)
tap_clean['group_4'] = tap_clean['group'].apply(lambda x: 1 if x == '4-SUNY CC' else 0)
tap_clean['group_5'] = tap_clean['group'].apply(lambda x: 1 if x == '5-INDEPENDENT' else 0)
tap_clean['group_6'] = tap_clean['group'].apply(lambda x: 1 if x == '6-BUS. DEGREE' else 0)
tap_clean['group_7'] = tap_clean['group'].apply(lambda x: 1 if x == '7-BUS. NON-DEG' else 0)
tap_clean['group_8'] = tap_clean['group'].apply(lambda x: 1 if x == '8-OTHER' else 0)
tap_clean['group_9'] = tap_clean['group'].apply(lambda x: 1 if x == '9-CHAPTER XXII' else 0)

np.sum(tap_clean['group_1']==1) + np.sum(tap_clean['group_2']==1) + np.sum(tap_clean['group_3']==1) + np.sum(tap_clean['group_4']==1) + np.sum(tap_clean['group_5']==1) + np.sum(tap_clean['group_6']==1) + np.sum(tap_clean['group_7']==1) + np.sum(tap_clean['group_8']==1) + np.sum(tap_clean['group_9']==1) 

190964

**Recipient Age Group**

The Recipient Age Group feature encodes for the age group of the applicant: 
* age 22 - 25
* age 26 - 35
* under age 22
* age 36 - 50
* over age 50

Again, like above, we one-hot encode for the age group.

In [49]:
# clean the age group and transform with one-hot encoding
print(tap_clean.groupby('age').size())
print(np.sum(tap_clean.groupby('age').size()))

age
age 22 - 25     46902
age 26 - 35     52913
age 36 - 50     36267
over age 50     15338
under age 22    39544
dtype: int64
190964


In [50]:
# one-hot encode for age group
tap_clean['age'] = tap_clean['age'].str.strip()

tap_clean['age_22'] = tap_clean['age'].apply(lambda x: 1 if x == 'age 22 - 25' else 0)
tap_clean['age_26'] = tap_clean['age'].apply(lambda x: 1 if x == 'age 26 - 35' else 0)
tap_clean['age_36'] = tap_clean['age'].apply(lambda x: 1 if x == 'age 36 - 50' else 0)
tap_clean['age_o50'] = tap_clean['age'].apply(lambda x: 1 if x == 'over age 50' else 0)
tap_clean['age_u22'] = tap_clean['age'].apply(lambda x: 1 if x == 'under age 22' else 0)

np.sum(tap_clean['age_22']==1) + np.sum(tap_clean['age_26']==1) + np.sum(tap_clean['age_36']==1) + np.sum(tap_clean['age_o50']==1) + np.sum(tap_clean['age_u22']==1)

190964

**TAP Award Schedule**

There are 3 award schedules: Dependent Schedule, Independent Schedule, or Married No Dependents Schedule
The TAP Award Schedule feature describes the type (out of 3) of award schedule the applicant is applying for:
* Dependent Schedule
* Independent Schedule
* Married No Dependents Schedule

Again, like above, we one-hot encode for the award schedule. 

In [51]:
# clean the award schedule column and transform with one-hot encoding
print(tap_clean.groupby('schedule').size())
print(np.sum(tap_clean.groupby('schedule').size()))

schedule
Dependent_Schedule       164602
Independent_Schedule      18675
Married_No_Dependents      7687
dtype: int64
190964


In [52]:
# one-hot encode for award schedule
tap_clean['schedule'] = tap_clean['schedule'].str.strip()

tap_clean['sched_dep'] = tap_clean['schedule'].apply(lambda x: 1 if x == 'Dependent_Schedule' else 0)
tap_clean['sched_ind'] = tap_clean['schedule'].apply(lambda x: 1 if x == 'Independent_Schedule' else 0)
tap_clean['sched_mar'] = tap_clean['schedule'].apply(lambda x: 1 if x == 'Married_No_Dependents' else 0)

sum(tap_clean['sched_dep']==1) + sum(tap_clean['sched_ind']==1) + sum(tap_clean['sched_mar']==1)

190964

**TAP Schedule Letter**

The TAP Schedule Letter feature defines the type of schedule the applicant is applying for, as defined by the TAP additional documentation.
* A, C, D, E, F, G, H, K, L, M, P, U, V

Again, like above, we one-hot encode for the schedule letter. 

In [53]:
# clean the award schedule column and transform with one-hot encoding
print(tap_clean.groupby('schedule letter').size())
print(np.sum(tap_clean.groupby('schedule letter').size()))

schedule letter
A     21080
C      7687
D     37241
E    101419
F        27
G        16
H       305
K      3086
L      1870
M     12461
P        77
U      4514
V      1181
dtype: int64
190964


In [54]:
# one-hot encode for award schedule letter
tap_clean['schedule letter'] = tap_clean['schedule letter'].str.strip()

tap_clean['letter_A'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'A' else 0)
tap_clean['letter_C'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'C' else 0)
tap_clean['letter_D'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'D' else 0)
tap_clean['letter_E'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'E' else 0)
tap_clean['letter_F'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'F' else 0)
tap_clean['letter_G'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'G' else 0)
tap_clean['letter_H'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'H' else 0)
tap_clean['letter_K'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'K' else 0)
tap_clean['letter_L'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'L' else 0)
tap_clean['letter_M'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'M' else 0)
tap_clean['letter_P'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'P' else 0)
tap_clean['letter_U'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'U' else 0)
tap_clean['letter_V'] = tap_clean['schedule letter'].apply(lambda x: 1 if x == 'V' else 0)


sum(tap_clean['letter_A']==1) + sum(tap_clean['letter_C']==1) + sum(tap_clean['letter_D']==1) + sum(tap_clean['letter_E']==1) + sum(tap_clean['letter_F']==1) + sum(tap_clean['letter_G']==1) + sum(tap_clean['letter_H']==1) +sum(tap_clean['letter_K']==1) + sum(tap_clean['letter_L']==1) + sum(tap_clean['letter_M']==1) + sum(tap_clean['letter_P']==1) + sum(tap_clean['letter_U']==1) + sum(tap_clean['letter_V']==1)  

190964

In [55]:
# delete columns that were one-hot-encoded for
tap_clean.drop('level', axis=1, inplace=True)
tap_clean.drop('group', axis=1, inplace=True)
tap_clean.drop('age', axis=1, inplace=True)
tap_clean.drop('schedule', axis=1, inplace=True)
tap_clean.drop('schedule letter', axis=1, inplace=True)

**Income by $1,000 Range**

The Income by \$1,000 feature is the most granular feature in the income dataset, returning the applicant's income to the nearest \$1,000.

In [56]:
def avg_income(str):
    if str=="0":
        return 0
    elif str.find("to") == -1:
        return 0
    else:
        str = str.replace(',', '')
        str = str.replace('$', '')
        i = str.find("to")
        num1 = int(str[0:i].strip())
        num2 = int(str[i+2:len(str)].strip())
        return (num1 + num2)/2.0

In [57]:
tap_clean['income_1'] = tap_clean['income_1'].str.strip()
tap_clean['avg_income'] = tap_clean['income_1'].apply(avg_income)

**TAP Recipient FTE**

There exist two empty data for this feature, both corresponding to headcounts of 1, so we replace the empty values with 0.75 (since 1 headcount can only be equal to either 0.5 or 1 FTE.)

In [58]:
mean_fte = 0.75
tap_clean['fte'] = tap_clean['fte'].apply(lambda x: 0.75 if np.isnan(x) else x)

In [59]:
tap_clean['dollars'] = tap_clean['dollars'].str.replace('$','')
tap_clean['dollars'] = tap_clean['dollars'].str.replace(',','')
tap_clean['dollars'] = tap_clean['dollars'].str.strip()
tap_clean['dollars'] = tap_clean['dollars'].astype(np.float64)

**TAP Award per FTE**

In [60]:
tap_clean['award_per_fte'] = tap_clean['dollars']/tap_clean['fte']

In [61]:
tap_clean.dtypes

year               int64
undergrad          int64
type               int64
dependent          int64
degree             int64
income_1          object
income_5          object
income_10         object
headcount          int64
fte              float64
dollars          float64
award_per_fte    float64
level_2            int64
level_4            int64
level_5            int64
level_g            int64
level_s            int64
group_1            int64
group_2            int64
group_3            int64
group_4            int64
group_5            int64
group_6            int64
group_7            int64
group_8            int64
group_9            int64
age_22             int64
age_26             int64
age_36             int64
age_o50            int64
age_u22            int64
sched_dep          int64
sched_ind          int64
sched_mar          int64
letter_A           int64
letter_C           int64
letter_D           int64
letter_E           int64
letter_F           int64
letter_G           int64


In [62]:
# finally, delete columns that are already encoded for (here, income_1, income_5, and income_10)
# delete columns that were one-hot-encoded for
tap_clean.drop('income_1', axis=1, inplace=True)
tap_clean.drop('income_5', axis=1, inplace=True)
tap_clean.drop('income_10', axis=1, inplace=True)

In [63]:
tap_clean

,year,undergrad,type,dependent,degree,headcount,fte,dollars,award_per_fte,level_2,...,letter_F,letter_G,letter_H,letter_K,letter_L,letter_M,letter_P,letter_U,letter_V,avg_income
0,2015,1,1,1,1,23,18.92,96170.20,5082.991543,0,...,0,0,0,0,0,0,0,0,0,1500.5
1,2015,1,1,0,1,10,6.20,11802.47,1903.624194,1,...,0,0,0,0,0,1,0,0,0,6500.5
2,2015,1,1,1,1,1,0.17,847.96,4988.000000,0,...,0,0,0,0,0,0,0,0,0,2500.5
3,2015,1,1,1,1,12,10.30,48704.45,4728.587379,1,...,0,0,0,0,0,0,0,0,0,12500.5
4,2015,1,1,0,1,2,1.51,4951.94,3279.430464,1,...,0,0,0,0,0,0,0,0,0,25500.5
5,2015,1,0,1,1,58,48.06,167056.86,3476.006242,0,...,0,0,0,0,0,0,0,0,0,23500.5
6,2015,1,1,0,1,1,0.50,1462.50,2925.000000,1,...,0,0,0,0,0,1,0,0,0,500.5
7,2015,1,1,1,1,26,23.35,118749.79,5085.644111,0,...,0,0,0,0,0,0,0,0,0,5500.5
8,2015,1,0,0,1,14,11.01,56055.55,5091.330609,0,...,0,0,0,0,0,0,0,0,0,2500.5
9,2015,1,1,0,1,1,1.01,502.58,497.603960,1,...,0,0,0,0,0,0,0,0,0,65500.5


In [64]:
tap_clean['fte'][97269]

0.75

In [65]:
tap_clean.drop('headcount', axis=1, inplace=True)
tap_clean.drop('fte', axis=1, inplace=True)
tap_clean.drop('dollars', axis=1, inplace=True)

In [66]:
# tap_clean.to_csv('TAP_data_numeric.csv')